In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

import pandas as pd
from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/batch_run/faith_template5/execution/Mistral-7B-Instruct-v0.2_rv16-ld0.05-bs1-p0-nta1-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
print(df.iloc[0].translated_prompt)

In [ ]:
print(df.iloc[0].linked_query)

In [ ]:
print(df.iloc[0].target_raw)


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

print(f"{tokenizer.unk_token=}")
print(f"{tokenizer.pad_token=}")
print(f"{tokenizer.unk_token_id=}")
print(f"{tokenizer.pad_token_id=}")

tokenizer.pad_token = tokenizer.unk_token

print(f"{tokenizer.unk_token=}")
print(f"{tokenizer.pad_token=}")
print(f"{tokenizer.unk_token_id=}")
print(f"{tokenizer.pad_token_id=}")

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/batch_run/democracy_template8/execution/Mistral-7B-Instruct-v0.2_rv32-ld0.05-bs1-p0-nta1-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
handpicked_data = df.loc[(~df['execution'].str.startswith("exception:")) & (~df['execution'].isnull()) & (df['execution'].map(len) > 0)].iloc[[1, 2, 3, 206, 207]]

In [ ]:
preds = tokenizer.batch_encode_plus([f"`sparql\n{i}`" for i in handpicked_data['translated_prompt'].to_list()], add_special_tokens=False, padding='longest', return_tensors='np')['input_ids'].tolist()
labels = tokenizer.batch_encode_plus([f"`sparql\n{i}`" for i in handpicked_data['target_template'].to_list()], add_special_tokens=False, padding='longest', return_tensors='np')['input_ids'].tolist()

In [ ]:
import json
from pathlib import Path

data = {
    "preds": preds,
    "labels": labels,
}

path = Path("sft_peft_compute_metrics_execute_ok.json")
path.write_text(json.dumps(data))

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset


df = load_dataset("../outputs/batch_run/lezgo/execution/Mistral-7B-Instruct-v0.2_rv16-ld0-bs1-p0-nta0-e3-template_engpeft-t0.2-topp0.95_executed.parquet.gzip")
df.head()

In [ ]:
import json
from pathlib import Path

df_out = df.head()

path = Path("evaluation_test.json")
path.write_text(df_out.to_json())

In [ ]:
import pandas as pd

data_gold = json.loads(Path("../outputs/batch_run/lezgo/preprocessed_gold.json").read_text())
df_gold = pd.read_json(data_gold['df_gold_eval'])
df_gold.head()

In [ ]:
df_gold_out = df_gold.head()

path = Path("evaluation_test_gold.json")
path.write_text(df_gold_out.to_json())

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

import pandas as pd

In [ ]:
dataset = load_dataset("../datasets/final_queries_v1.7.json")
dataset.head()

In [ ]:
df_out = dataset[2549:2553]
df_out.head()

In [ ]:
df_out_v2 = df_out.reset_index(drop=True)
df_out_v2.head()

In [ ]:
path = Path("templatize_test.json")
path.write_text(df_out_v2.to_json())

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl")
df.head()

In [ ]:
entry = df.iloc[0]
for c in df.columns:
    print(c)
    print(entry[c])
    print()

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from data_utils import load_dataset

In [ ]:
df = load_dataset("../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl")
df.head()

In [ ]:
print(df.iloc[2].target_template)

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from datasets import load_dataset
from sft_peft import format_prompt
import sft_peft
from prompts_template import BASE_MISTRAL_TEMPLATE, BASE_BASIC_INSTRUCTION
from libwikidatallm.TemplateLLMQuerySender import TemplateLLMQuerySender

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token
collator = DataCollatorForCompletionOnlyLM(response_template="[/INST]", tokenizer=tokenizer)
datasets = load_dataset("pandas", data_files=
                          {
                              "valid": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_valid.pkl",
                              "train": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_train.pkl",
                              "test": "../outputs/dataset_pipeline/fq17_3/fq17_3-split_test.pkl",
                          })

In [4]:
datasets

DatasetDict({
    valid: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 129
    })
    train: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 1923
    })
    test: Dataset({
        features: ['basic_input', 'templated_input', 'target_raw', 'target_template', 'index'],
        num_rows: 513
    })
})

In [5]:
# Taken from SFTTrainer(Trainer) _prepare_non_packed_dataloader method
def _prepare_non_packed_dataloader(
    tokenizer,
    dataset,
    dataset_text_field,
    max_seq_length,
    formatting_func=None,
    add_special_tokens=True,
    remove_unused_columns=True,
):
    use_formatting_func = formatting_func is not None and dataset_text_field is None
    # self._dataset_sanity_checked = False

    # Inspired from: https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt
    def tokenize(element):
        outputs = tokenizer(
            element[dataset_text_field] if not use_formatting_func else formatting_func(element),
            add_special_tokens=add_special_tokens,
            truncation=True,
            padding=False,
            max_length=max_seq_length,
            return_overflowing_tokens=False,
            return_length=False,
        )

        # if use_formatting_func and not self._dataset_sanity_checked:
        #     if not isinstance(formatting_func(element), list):
        #         raise ValueError(
        #             "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
        #         )
        #     else:
        #         self._dataset_sanity_checked = True

        return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

    signature_columns = ["input_ids", "labels", "attention_mask"]

    extra_colmuns = list(set(dataset.column_names) - set(signature_columns))

    # if not remove_unused_columns and len(extra_colmuns) > 0:
    #     warnings.warn(
    #         "You passed `remove_unused_columns=False` on a non-packed dataset. This might create some issues with the default collator and yield to errors. If you want to "
    #         f"inspect dataset other columns (in this case {extra_colmuns}), you can subclass `DataCollatorForLanguageModeling` in case you used the default collator and create your own data collator in order to inspect the unused dataset columns."
    #     )

    tokenized_dataset = dataset.map(
        tokenize,
        batched=True,
        remove_columns=dataset.column_names if remove_unused_columns else None,
        num_proc=1,
        batch_size=32,
    )

    return tokenized_dataset

In [6]:
template = BASE_MISTRAL_TEMPLATE
sft_peft.templater = TemplateLLMQuerySender(None, template, start_seq='[', end_seq=']')
sft_peft.input_column = "basic_input"
sft_peft.target_column = "target_raw"

In [7]:
tokenized_dataset = _prepare_non_packed_dataloader(tokenizer, datasets['test'], None, None, format_prompt)

In [20]:
tokenized_dataset[]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 513
})

In [ ]:
collator(tokenized_dataset, return_tensors="np")

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from sft_peft import main, parse_args
import argparse


In [ ]:
args = parse_args([
    "--model", "mistralai/Mistral-7B-Instruct-v0.2",
    "--train-data", "../outputs/dataset_pipeline/fq17_3/fq17_3-split_train.pkl",
    "--target-column", "target_raw",
    "--input-column", "basic_input",
    "--valid-data", "../outputs/dataset_pipeline/fq17_3/fq17_3-split_valid.pkl",
    "--start-tag", "[query]",
    "--end-tag", "[/query]",
    "--rvalue", "32",
    "--lora-dropout", "0",
    "--batch-size", "1",
    "--gradient-accumulation", str(4),
    "--packing", "0",
    "--neft-tune-alpha", "0",
    "--epochs", "1",
    "--output", ".",
    "--save-name", "test_debug",
    "--run-name", f"test_debug",
    "--random-seed", "42"
])

In [ ]:
main(args)

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules").absolute().__str__())

from transformers import AutoTokenizer


In [2]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.unk_token

In [3]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
print_tokens_with_ids("[/INST]")
print_tokens_with_ids("[/INST] [query]")
print_tokens_with_ids("[/INST] [query]")

[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793)]
[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793), ('▁[', 733), ('query', 3385), (']', 28793)]
[('▁[', 733), ('/', 28748), ('INST', 16289), (']', 28793), ('▁[', 733), ('query', 3385), (']', 28793)]
